# Baseline model - Logistic Regression

## Imports and Load Data

In [27]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform

from sklearn.metrics import f1_score, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import auc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn import preprocessing


In [28]:
data = pd.read_csv('../data/utlysningar_preprocessed_stemming')

In [31]:
data

,Title En,Description En,Research fields
0,"Judith Jack Halberstam, Columbia University, N...",jack halberstam legal name judith legal gender...,5
1,"Nomination of Professor Martin Shepperd, Brune...",professor martin john shepperd phd dept comput...,4
2,Characterisation and effects of micro and nano...,start recent develop new experiment techniqu c...,1
3,Moral Vagueness in a Mind-Independent World,the main object project show central hypothesi...,2
4,Bearbetning av organosolvfraktioner för funkti...,the global demand renew degrad sustain materi ...,1
...,...,...,...
7138,Spatial Omics Enable Improved Pathophysiology-...,we propos innov interdisciplinari research env...,3
7139,A New Interdisciplinary Framework for Studying...,the program aim establish bold interdisciplina...,5
7140,SweDigArch – The Swedish National Infrastructu...,swedigarch construct run swedish nation infras...,5
7141,Design after progress: reimagining design hist...,design tie industrialis progress made possibl ...,2


In [30]:
# Encode the labels

le = preprocessing.LabelEncoder()
le.fit(data['Research fields'])
data['Research fields'] = le.transform(data['Research fields'])

In [32]:
X_train,X_test,Y_train,Y_test = train_test_split(data['Description En'],data['Research fields'],test_size = 0.3,random_state=2)

## Feature Extraction & Embedding - Bag of Words

In [33]:
count_vect = CountVectorizer(ngram_range=(1,2))

X_train_bow = count_vect.fit_transform(X_train) 
X_test_bow = count_vect.transform(X_test)

In [34]:
X_train_bow = preprocessing.normalize(X_train_bow)
X_test_bow = preprocessing.normalize(X_test_bow)

## Fine tuning 

In [36]:
tuned_parameters = { 'C': [10**-4, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**4],
              'penalty':['l1','l2']}
model_precision = RandomizedSearchCV(LogisticRegression(), tuned_parameters,
                      scoring = "f1"
                     )
model_precision.fit(X_train_bow, Y_train)

print("Best C and penalty",model_precision.best_params_)
print("precision on train data",model_precision.best_score_*100)

Best C and penalty {'penalty': 'l2', 'C': 10}
precision on train data 75.58
